In [1]:
import os
from PIL import Image
import torch
import numpy as np

In [2]:
path = "/net/ens/am4ip/datasets/project-dataset"
rainyImagesPath = path + '/rainy_images'
rainySsegPath = path + '/rainy_sseg'
sunnyImagesPath = path + '/sunny_images'
sunnySsegPath = path + '/sunny_sseg'

In [ ]:
model_weights_path = '/net/cremi/mhajnal/espaces/travail/am4ipProject/DONE/denoisingModel.pth'
model = torch.load(model_weights_path)

# Run test dataset through network
model.eval()

/tmp/cache-mhajnal/ipykernel_1351256/1769471316.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(model_weights_path)


UNet(
  (upfeature): FeatureMapBlock(
    (conv): Conv2d(1, 32, kernel_size=(1, 1), stride=(1, 1))
  )
  (contracts): Sequential(
    (0): DownDouble(
      (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (activation): LeakyReLU(negative_slope=0.2)
      (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): DownDouble(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (activation): LeakyReLU(negative_slope=0.2)
      (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (expands): Sequential(
    (0): UpDouble(
      (upsample): Upsample(scale_factor=2.0, mode='bilinear')
      (conv1): Conv2d(128, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2)

In [4]:
num_images = 3779

# Initialize a list to store the images and filenames
images_list = []
filenames_list = []

# Iterate over the first `num_images` files in the folder
for i, filename in enumerate(sorted(os.listdir(sunnyImagesPath))):
    if filename.endswith(".png") or filename.endswith(".jpg"):  # Add other extensions if necessary
        # Load the image
        img_path = os.path.join(sunnyImagesPath, filename)
        image = Image.open(img_path)
        resized_image = image.resize((320, 480))
        grayscale_image = resized_image.convert("L")
        img = np.array(grayscale_image)

        images_list.append(np.array(img))  # Convert to numpy array and append to list
        filenames_list.append(filename)  # Save the filename
        if len(images_list) >= num_images:  # Stop after reading `num_images`
            break

# Stack the images to create a 3D numpy array
images_array = np.stack(images_list, axis=0)  # Shape: (num_images, n, n)

print(f"Images array shape: {images_array.shape}")
print(f"Processed filenames: {filenames_list}")

Images array shape: (3779, 480, 320)
Processed filenames: ['00009146.jpg', '00009147.jpg', '00009148.jpg', '00009149.jpg', '00009150.jpg', '00009151.jpg', '00009152.jpg', '00009153.jpg', '00009154.jpg', '00009155.jpg', '00009156.jpg', '00009157.jpg', '00009158.jpg', '00009159.jpg', '00009160.jpg', '00009161.jpg', '00009162.jpg', '00009163.jpg', '00009164.jpg', '00009165.jpg', '00009166.jpg', '00009167.jpg', '00009168.jpg', '00009169.jpg', '00009170.jpg', '00009171.jpg', '00009172.jpg', '00009173.jpg', '00009174.jpg', '00009175.jpg', '00009176.jpg', '00009177.jpg', '00009178.jpg', '00009179.jpg', '00009180.jpg', '00009181.jpg', '00009182.jpg', '00009183.jpg', '00009184.jpg', '00009185.jpg', '00009186.jpg', '00009187.jpg', '00009188.jpg', '00009189.jpg', '00009190.jpg', '00009191.jpg', '00009192.jpg', '00009193.jpg', '00009194.jpg', '00009195.jpg', '00009196.jpg', '00009197.jpg', '00009198.jpg', '00009199.jpg', '00009200.jpg', '00009201.jpg', '00009202.jpg', '00009203.jpg', '00009204.jpg

In [5]:
video1 = np.empty_like(images_array)
for i in range(len(images_array)):
    testdata = images_array[i]
    
    # Convert dataset in tensor for prediction purposes
    torch_testdata = torch.from_numpy(np.expand_dims(np.expand_dims(testdata,axis=0),axis=0)).float()
    
    
    test_prediction = model(torch_testdata.to("cuda"))
    video1[i] = test_prediction.cpu().detach().numpy()[0][0]

In [6]:
video1.shape

(3779, 480, 320)

In [7]:
output_folder = '/net/cremi/mhajnal/espaces/travail/am4ipProject/sunnyDenoised'
os.makedirs(output_folder, exist_ok=True)

for idx, frame in enumerate(video1):
    output_path = os.path.join(output_folder, filenames_list[idx])
    Image.fromarray(frame).save(output_path)

--- rain

In [4]:
num_images = 3642

# Initialize a list to store the images
images_list = []
filenames_list = []

# Iterate over the first `num_images` files in the folder
for i, filename in enumerate(sorted(os.listdir(rainyImagesPath))):
    if filename.endswith(".png") or filename.endswith(".jpg"):  # Add other extensions if necessary
        # Load the image
        img_path = os.path.join(rainyImagesPath, filename)
        image = Image.open(img_path)
        resized_image = image.resize((320, 480))
        grayscale_image = resized_image.convert("L")
        img = np.array(grayscale_image)

        images_list.append(np.array(img))  # Convert to numpy array and append to list
        filenames_list.append(filename)  # Save the filename
        if len(images_list) >= num_images:  # Stop after reading `num_images`
            break

# Stack the images to create a 3D numpy array
images_array = np.stack(images_list, axis=0)  # Shape: (10, n, n)

print(f"Images array shape: {images_array.shape}")
print(f"Processed filenames: {filenames_list}")

Images array shape: (3642, 480, 320)
Processed filenames: ['00053921.png', '00053922.png', '00053923.png', '00053924.png', '00053925.png', '00053926.png', '00053927.png', '00053928.png', '00053929.png', '00053930.png', '00053931.png', '00053932.png', '00053933.png', '00053934.png', '00053935.png', '00053936.png', '00053937.png', '00053938.png', '00053939.png', '00053940.png', '00053941.png', '00053942.png', '00053943.png', '00053944.png', '00053945.png', '00053946.png', '00053947.png', '00053948.png', '00053949.png', '00053950.png', '00053951.png', '00053952.png', '00053953.png', '00053954.png', '00053955.png', '00053956.png', '00053957.png', '00053958.png', '00053959.png', '00053960.png', '00053961.png', '00053962.png', '00053963.png', '00053964.png', '00053965.png', '00053966.png', '00053967.png', '00053968.png', '00053969.png', '00053970.png', '00053971.png', '00053972.png', '00053973.png', '00053974.png', '00053975.png', '00053976.png', '00053977.png', '00053978.png', '00053979.png

In [5]:
video1 = np.empty_like(images_array)
for i in range(len(images_array)):
    testdata = images_array[i]
    
    # Convert dataset in tensor for prediction purposes
    torch_testdata = torch.from_numpy(np.expand_dims(np.expand_dims(testdata,axis=0),axis=0)).float()
    
    
    test_prediction = model(torch_testdata.to("cuda"))
    video1[i] = test_prediction.cpu().detach().numpy()[0][0]

In [6]:
output_folder = '/net/cremi/mhajnal/espaces/travail/am4ipProject/rainyDenoised'
os.makedirs(output_folder, exist_ok=True)

for idx, frame in enumerate(video1):
    output_path = os.path.join(output_folder, filenames_list[idx])
    Image.fromarray(frame).save(output_path)